In [26]:
%pip install fbprophet

Defaulting to user installation because normal site-packages is not writeable
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [40 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/fbprophet
      creating build/lib/fbprophet/stan_model
      Importing plotly failed. Interactive plots will not work.
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-zue846on/fbprophet_a788345e521f4abeae0977a0068889cf/setup.py", line 122, in <module>
          setup(
        File "/usr/lib/python3/dist-packages/setuptools/__init__.py", line 153, in setup
          return distutils.core.setup(**attrs)
        File "/u

In [27]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
#import plotly.express as px  

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

df = pd.read_csv('resources/transactions_1.csv')

df

,time,status,count
0,00h 00,approved,9
1,00h 00,denied,6
2,00h 00,refunded,1
3,00h 01,denied,8
4,00h 01,approved,13
...,...,...,...
4229,23h 59,processing,4
4230,23h 59,denied,1
4231,23h 59,backend_reversed,5
4232,23h 59,approved,10


In [28]:
df['time'] = pd.to_datetime(df['time'], format='%Hh %M').dt.time
df

,time,status,count
0,00:00:00,approved,9
1,00:00:00,denied,6
2,00:00:00,refunded,1
3,00:01:00,denied,8
4,00:01:00,approved,13
...,...,...,...
4229,23:59:00,processing,4
4230,23:59:00,denied,1
4231,23:59:00,backend_reversed,5
4232,23:59:00,approved,10


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4234 entries, 0 to 4233
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    4234 non-null   object
 1   status  4234 non-null   object
 2   count   4234 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 99.4+ KB
